In [1]:
#!python -m spacy download en_core_web_sm

In [2]:
from booknlp.booknlp import BookNLP


using device cpu


In [14]:

model_params={
		"pipeline":"entity,quote,supersense,event,coref",
		"model":"small"

	}

booknlp=BookNLP("en", model_params)

inputFile="./crusoe.txt"
outputDir="./out/"
book_id="crusoe"

booknlp.process(inputFile, outputDir, book_id)

{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'small'}
--- startup: 1.596 seconds ---
--- spacy: 16.864 seconds ---
--- entities: 269.007 seconds ---
--- quotes: 0.370 seconds ---
--- attribution: 27.752 seconds ---
--- name coref: 0.701 seconds ---
--- coref: 195.858 seconds ---
--- TOTAL (excl. startup): 511.853 seconds ---, 139719 words


In [ ]:
import pandas as pd
df = pd.read_csv("./out/crusoe.supersense")

In [ ]:
df

## 断点续传版

In [ ]:
import os
from tqdm import tqdm
from booknlp.booknlp import BookNLP

model_params = {
    "pipeline": "entity,quote,supersense,event,coref",
    "model": "small"
}

booknlp = BookNLP("en", model_params)
inputDir = "../corpus/19c_last50"
outputDir = "./output19/"

# 确保输出文件夹存在
os.makedirs(outputDir, exist_ok=True)

# 获取输入文件夹下所有 txt 文件列表
txt_files = [filename for filename in os.listdir(inputDir) if filename.endswith(".txt")]

# 获取输出文件夹下已处理过的文件列表
processed_files = [filename[:-8] for filename in os.listdir(outputDir) if filename.endswith(".tokens")]

# 筛选出尚未处理的文件列表
unprocessed_files = [filename for filename in txt_files if filename[:-4] not in processed_files]

# 使用 tqdm 包装器添加进度条
for filename in tqdm(unprocessed_files, desc="Processing files"):
    book_id = os.path.splitext(filename)[0]  # 使用文件名作为 book_id
    inputFilePath = os.path.join(inputDir, filename)
    
    # 确认文件是否已经完整处理过
    fully_processed = all([os.path.exists(os.path.join(outputDir, book_id + ext)) for ext in [".book.html", ".book", ".entities", ".quotes", ".supersense", ".tokens"]])
    
    if not fully_processed:
        print("Processing file:", filename)  # 输出提示信息
        booknlp.process(inputFilePath, outputDir, book_id)
    else:
        print("File already processed:", filename)



using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'small'}
--- startup: 3.744 seconds ---


Processing files:   0%|                                                        | 0/50 [00:00<?, ?it/s]

File already processed: 1897_Liza of Lambeth.txt
File already processed: 1897_St. Ives_Being The Adventures of a French Prisoner in England.txt
Processing file: 1897_The Beetle_A Mystery.txt
--- spacy: 34.588 seconds ---
--- entities: 314.819 seconds ---
--- quotes: 0.420 seconds ---
--- attribution: 213.167 seconds ---
--- name coref: 1.524 seconds ---
--- coref: 258.067 seconds ---


Processing files:   6%|██▋                                          | 3/50 [13:44<3:35:22, 274.95s/it]

--- TOTAL (excl. startup): 824.678 seconds ---, 137597 words
Processing file: 1897_The Blood of the Vampire.txt
--- spacy: 36.619 seconds ---


In [ ]:
metadata

## 对booknlp的代码解析：
english_booknlp.py
这段代码是一个Python类 EnglishBookNLP 的定义，它使用了一系列自然语言处理（NLP）技术和模型来分析和处理英文书籍文本。这个类提供了一个完整的管道，用于执行多种文本分析任务，包括实体识别、共指解析、引语分析和性别推断。下面是代码的主要功能和组件的详细解释：

初始化 (__init__ 方法):

载入 SpaCy 的英文模型（默认为 en_core_web_sm），但可以通过参数更改。
定义了一个性别类别列表，用于性别推断。
根据提供的参数，确定模型的路径和大小（"big", "small" 或 "custom"）。
下载并保存所需的模型文件到本地路径。
初始化各种处理管道，根据参数启用或禁用特定的功能（实体、事件、共指、引语、性别推断）。
获取句法信息 (get_syntax 方法):

分析文本中的实体，确定它们的句法角色，如施事者（agent）、受事者（patient）、所有格（possessive）等。
收集和整理实体的提及信息，包括专有名词、普通名词和代词。
处理文本 (process 方法):

读取输入文件并进行 SpaCy 分词和标注。
根据初始化时设置的管道参数，执行实体识别、事件识别、共指解析、引语分析等任务。
对于每个识别出的实体，确定其在文本中的提及，并进行性别推断。
将分析结果保存到输出文件夹中的多个文件中，包括实体、共指、引语和书籍的JSON文件，以及一个HTML文件，用于可视化文本中的命名实体和主要实体。
辅助功能:

使用 urllib.request.urlretrieve 方法下载所需的模型文件。
使用 os 和 pathlib 模块处理文件和目录路径。
使用 json 模块保存分析结果为JSON文件。
使用 html 模块和 escape 函数来转义HTML中的文本内容。
整体而言，这段代码是一个复杂的NLP处理管道，旨在从英文书籍文本中提取丰富的语言特征和结构信息。它结合了多个预训练模型和自定义的解析算法，以支持对文本内容的深入分析。

这段代码使用了多个预训练模型，每个模型都用于特定的自然语言处理任务。以下是代码中使用的预训练模型及其用途：

Spacy 模型 (en_core_web_sm):

用于基础的文本处理，包括分词、词性标注和依赖关系解析。
默认情况下，该模型被禁用了命名实体识别（NER），因为它将被后续的实体识别模型替代。
LitBankEntityTagger:

用于识别文本中的实体，特别是人名、地点、组织等。
这是一个基于LitBank标注数据集训练的实体识别器。
LitBankCoref:

用于执行共指解析，即识别文本中提到的不同表达方式实际上指代同一个实体的情况。
它使用预训练的模型来识别和链接提到的人名、代词和其他提及。
QuotationAttribution:

用于识别文本中的引语，并将其归属于正确的说话者。
这个模型分析文本中的引语，并尝试确定说话者的身份。
NameCoref:

用于处理名字的共指解析，将文本中的名字与对应的实体进行关联。
它还处理了叙述者的名字，以及文本中提到的其他专有名词。
GenderEM:

用于推断文本中提及的实体的性别。
它使用预训练的模型来分析文本中的性别相关提及，并推断出每个实体的性别。
SpacyPipeline:

这是一个自定义的管道，它将上述模型和组件整合到一个处理流程中，以便对文本进行综合分析。
这些模型和组件共同工作，提供了一个全面的NLP解决方案，用于分析和理解英文书籍文本的结构和含义。通过这种方式，EnglishBookNLP 类能够提取出文本中的实体、事件、共指关系、引语和性别信息，并将这些信息以结构化的形式保存和展示。

## bert_qa.py的代码解析

这段代码定义了一个名为 QuotationAttribution 的类，它用于将文本中的引语（直接引语）归属于说话者。这个类使用了基于BERT的模型 BERTSpeakerID 来处理引语，并确定每个引语的说话者。以下是代码的主要组成部分和它们的功能：

初始化 (__init__ 方法):

载入预训练的 BERTSpeakerID 模型，这个模型专门用于识别引语的说话者。
模型文件路径通过正则表达式处理，以确保模型名称的正确性。
模型被加载到可用的GPU或CPU上，并设置为评估模式（eval）。
标记引语 (tag 方法):

这个方法接收引语列表、实体列表和分词后的文本（tokens）作为输入。
它首先创建一个用于存储引语归因的列表 attributions。
然后，它为每个实体创建一个映射，以便快速查找实体的索引。
接下来，它调用 get_representation 方法来获取引语的文本表示，包括相关的元数据和位置信息。
使用 BERTSpeakerID 模型对每个引语进行预测，确定说话者，并更新 attributions 列表。
获取表示 (get_representation 方法):

这个方法负责准备输入数据，以便模型可以处理。
它首先标记引语中的每个单词，并为特殊标记（如段落开始 [PAR] 和引语 [QUOTE]）创建映射。
然后，它为每个引语创建一个上下文窗口，包括前后一定数量的单词，以提供足够的上下文信息。
对于每个引语，它生成一个文本列表 toks，其中包含原始文本和特殊标记。
它还创建了一个元数据列表 metas，其中包含每个引语的标签和位置信息。
最后，它返回文本表示、元数据、位置信息、全局实体位置和引语索引列表。
辅助功能:

convert_word 函数用于处理单词的大小写，并添加特殊标记。
get_base 函数用于处理引语链，即一个引语内嵌套另一个引语的情况。
整体而言，这段代码的目的是将文本中的直接引语归属于正确的说话者。它通过分析引语的上下文和实体信息，使用预训练的BERT模型来预测每个引语的说话者。这种方法在处理对话文本和引语归因任务时非常有用。

## 模型下载的解释
english_booknlp.py
home = str(Path.home())
			modelPath=os.path.join(home, "booknlp_models")
			if "model_path"  in model_params:			
				modelPath=model_params["model_path"]

			if not Path(modelPath).is_dir():
				Path(modelPath).mkdir(parents=True, exist_ok=True)
解释：
这段代码的目的是确定模型文件的存储路径，并确保该路径所在的目录存在。下面是对这段代码的逐行解释：

home = str(Path.home()):
这行代码使用 Path.home() 函数获取当前用户的主目录（home directory）的路径，并将其转换为字符串格式。例如，在Windows上可能是 C:\Users\YourUsername，在Linux或macOS上可能是 /home/YourUsername。

modelPath = os.path.join(home, "booknlp_models"):
这行代码使用 os.path.join() 函数将用户的主目录路径（home）与子目录名 "booknlp_models" 连接起来，形成一个完整的目录路径。这通常是用于存放模型文件的默认路径。

if "model_path" in model_params::
这行代码检查一个名为 model_params 的字典（可能是从配置文件或用户输入中获取的参数）中是否存在一个键 "model_path"。如果存在，这意味着用户可能已经指定了一个自定义的模型存储路径。

modelPath = model_params["model_path"]:
如果上一步中的条件为真，这行代码将 modelPath 变量设置为 model_params 字典中 "model_path" 键对应的值。这样，模型文件将被存储在用户指定的路径而不是默认路径。

if not Path(modelPath).is_dir()::
这行代码检查 modelPath 变量所指向的路径是否已经是一个目录。如果不是，Path(modelPath).is_dir() 将返回 False。

Path(modelPath).mkdir(parents=True, exist_ok=True):
这行代码使用 Path 对象的 mkdir() 方法创建一个新的目录。parents=True 参数表示如果父目录不存在，也会一并创建。exist_ok=True 参数表示如果目录已经存在，不会抛出异常。这样，无论目标目录是否存在，代码都会确保 modelPath 所指向的目录被创建，而不会引发错误。

总的来说，这段代码负责确定和创建模型文件的存储路径，无论是使用默认路径还是用户自定义的路径，并确保该路径存在。

其中，entities,coref,speaker的模型下载指向Batmman自己的网页：
https://people.ischool.berkeley.edu/~dbamman/booknlp_models/

代码首先检查 model_params 字典中的 "model" 键来确定用户想要使用哪种模型："big"、"small" 或 "custom"。
每种模型类型对应一组不同的模型文件，这些文件用于不同的NLP任务。
大模型 ("big"):

如果选择的是大模型，代码将使用预定义的变量名（entityName、corefName、quoteAttribName）来指定实体识别、共指解析和引语归属模型的文件名。
然后，代码使用 os.path.join 将模型文件名与之前确定的 modelPath 路径结合起来，形成完整的文件路径（self.entityPath、self.coref_model、self.quoteAttribModel）。
如果相应的文件在本地路径中不存在（Path(self.entityPath).is_file() 返回 False），代码将打印出正在下载的文件名，并使用 urllib.request.urlretrieve 函数从指定的URL下载模型文件到本地路径。
小模型 ("small"):

与大模型类似，小模型也有对应的模型文件，但可能是更小或优化过的版本，适用于资源受限的环境。
同样的下载和路径设置逻辑被应用于小模型。
自定义模型 ("custom"):

如果用户选择自定义模型，代码将直接使用 model_params 字典中提供的自定义路径来获取模型文件路径，而不是从URL下载。这时调用的python代码为：
model_params={
		"pipeline":"entity,quote,supersense,event,coref",
		"model":"custom"

	}

booknlp=BookNLP("en", model_params)
对应的源代码如下：

			elif model_params["model"] == "custom":
				self.entityPath=model_params["entity_model_path"]
				self.coref_model=model_params["coref_model_path"]
				self.quoteAttribModel=model_params["quote_attribution_model_path"]
                
可见，"model":"custom"时需要传递参数：
"entity_model_path":"C:/Users/huihf/hui_models/entities_google_bert_uncased_L-4_H-256_A-4-v1.0.model"
"coref_model_path":"C:/Users/huihf/hui_models/coref_google_bert_uncased_L-2_H-256_A-4-v1.0.model"
"quote_attribution_model_path":"C:/Users/huihf/hui_models/speaker_google_bert_uncased_L-8_H-256_A-4-v1.0.1.model"

运行代码，仍然会从huggingface下载Bert模型，位于 C:\Users\huihf\.cache\huggingface\hub，分别是3个：
models--google--bert_uncased_L-8_H-256_A-4
models--google--bert_uncased_L-4_H-256_A-4
models--google--bert_uncased_L-2_H-256_A-4
下面分别有复杂的文件结构，共计大小135M

### 总结：
对于booknlp的windows用户来说，问题：
1。 不能从作者个人主页下载模型（实际上是不能保存），这是文件夹目录的斜杠问题，报错信息:.../...\...   解决办法：自己可以从作者主页下载好之后建立那三个文件夹entities, coref, speaker。 或者在py文件夹中修改os.join的斜杠改为两个斜杠

对于国内用户的问题：不能从huggingface下载Bert模型
解决：自行下载，放到目录：C:\Users\huihf\.cache\huggingface\hub 上面有

程序本身的运行问题utf8：进去english_booknlp.py修改open(filename)加上encoding="utf8"即可





## bert_qa.py  
这个专门负责调用bert模型进行引语分析，本身不包含任何直接的下载任务。它定义了一个名为 QuotationAttribution 的类，该类使用 BERTSpeakerID 模型来进行引语归属任务，即确定文本中引语的说话者。代码中的 BERTSpeakerID 模型是一个基于BERT的预训练模型，专门用于此任务。然而，代码中有一个部分涉及到模型文件的加载：

self.model = BERTSpeakerID(base_model=base_model)
self.model.load_state_dict(torch.load(modelFile, map_location=device))
在这里，torch.load 函数用于加载指定路径 modelFile 的模型状态字典。这个函数不会自动下载模型文件；它假设模型文件已经存在于给定的路径中。如果模型文件不存在，torch.load 将抛出一个错误。

如果你想使用这段代码，你需要确保已经下载了 BERTSpeakerID 模型文件，并提供了正确的文件路径。如果你没有模型文件，你需要从Hugging Face模型库或其他来源下载相应的预训练模型，并将其保存到本地文件系统中，然后在创建 QuotationAttribution 类的实例时提供该路径。

modelFile 参数是包含预训练 BERTSpeakerID 模型的文件路径。
device 变量根据是否有可用的GPU来决定模型应该运行在CPU还是GPU上。如果GPU可用，模型将使用GPU以加速计算；否则，使用CPU。
base_model 变量通过正则表达式处理 modelFile 来获取模型的基础名称，这通常是一个Hugging Face模型库中的模型名称。
self.model 是 BERTSpeakerID 类的一个实例，它使用 base_model 作为参数来初始化。
self.model.load_state_dict(torch.load(modelFile, map_location=device)) 这行代码加载预训练模型的状态字典，并将其映射到当前设备（CPU或GPU）。
self.model.to(device) 将模型移动到之前确定的设备上。
self.model.eval() 将模型设置为评估模式，这是为了确保在推理时模型的行为是正确的（例如，停用dropout层）。